

## Applying Sentiment analysis to movie review using Kaggle labeled dataset.

https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data

### Process unclassified phrases using saved Sentiment module and trained data


## Part 3 - Process phrases using Sentiment Model
__Summary__:  

Part 3 is a brief.  The sentiment model saved in part 2 is used to score unclassified phrases. In this case the test phrases provided by Kaggle.    


1.  Create a processed phrase table in Postgres DB.
2.  Incrementally process each phrase using saved sentiment module.  
---

In [8]:
#!/usr/local/bin/python3.6
#import logging
import os
import pyodbc
import psycopg2
import psycopg2.extras
import numpy as np
#import pandas as pd
#import pandas.io.sql as psql
from sqlalchemy import create_engine

In [2]:
#postgres authentication
user = "alexp"
password = "secret"
host = "pg_db"
port = "5432"
database = "priv_workspace"

In [3]:
#view pg environment
try:
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")
    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

{'user': 'alexp', 'dbname': 'priv_workspace', 'host': 'pg_db', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '1', 'krbsrvname': 'postgres'} 

You are connected to -  ('PostgreSQL 10.5 (Debian 10.5-1.pgdg90+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 6.3.0-18+deb9u1) 6.3.0 20170516, 64-bit',) 

PostgreSQL connection is closed


### 1.  Create a processed phrase table in Postgres DB.

In [30]:
#create total count of documents (phrases to be processed) incrementally process and write each document
sql1 = '''SELECT COUNT(*) FROM "sent_test_tble"'''

In [46]:
###create processedPhrasesTable table
processedPhrasesTable = '''CREATE TABLE IF NOT EXISTS "processed_phrases"(
    "phraseid" INTEGER REFERENCES sent_movie_test(phraseid), "sentenceid" INTEGER, 
 "rowno" INT,"sentiment" TEXT, "sent_score" NUMERIC (5,2)
);'''

### 2.  Incrementally process each phrase using saved sentiment module.

In [48]:
import sentiment_mod as s
#score phrases for sentiment and write to pg db
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

try:
    connection = psycopg2.connect(user = user,
                                  password = password,
                                  host = host,
                                  port = port,
                                  database = database)
    cursor = connection.cursor()
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    engine_string = "postgresql://{}:{}@{}:{}/{}".format(user, password, host, port, database)
    engine = create_engine(engine_string)
    
    #get total documents to be processed   
    cursor.execute(sql1)
    fetch = cursor.fetchall()
    totalDocuments = fetch[0][0]
    print('retrieved total documents ', fetch[0][0])
    
    #create processed table if doesnt already exist
    cursor.execute(processedPhrasesTable)
    
    for i in range(totalDocuments):
        row = i +1
        processRow = '''select * from (SELECT a.phraseid, a.sentenceid, a.phrase, 
        row_number() over (order by phrase)
        FROM "sent_test_tble" a ) b 
        where b.row_number = '{}' '''.format(row)
        #print(processRow)
        cursor.execute(processRow)
        fetch = cursor.fetchall()
        sentiment = s.sentiment(fetch[0][2])
        
        #print('retrieved total documents ', sentiment)
        #phraseid, sentenceid, rowid, sentiment, sent score
        processedPhrase = fetch[0][0:1] + fetch[0][1:2] + fetch[0][3:4] + sentiment
        sql='''insert into processed_phrases values({}, {}, {}, '{}', {})'''.format(processedPhrase[0], processedPhrase[1],
                                                            processedPhrase[2], processedPhrase[3],
                                                              processedPhrase[4])
        #insert processed documents
        cursor.execute(sql)
        #print('inserted row')
    print('inserted ', row,  ' rows into processed_phrases table')
    
    ##     
#     #phraseid, sentenceid, rowid, sentiment, sent score
#     processedPhrase = fetch[0][0:1] + fetch[0][1:2] + fetch[0][3:4] + sentiment
#     #print('processed insert ', processedPhrase)
    #

except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while creating PostgreSQL table", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

retrieved total documents  3310
inserted  3310  into processed_phrases table
PostgreSQL connection is closed
